In [21]:
import cv2
import os

# Define parameters
number_of_classes = 5  # Adjust based on your dataset
class_images = 1000  # Number of images per class
data_dir = "data"  # Directory to store the captured images

# Initialize webcam
cap = cv2.VideoCapture(0)

for j in range(number_of_classes):
    # Create directory for each class
    class_dir = os.path.join(data_dir, str(j))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    print(f"Collecting data for class {j}. Press 'm' to start capturing.")
    while True:
        ret, frame = cap.read()
        cv2.putText(frame, "Press 'm' to capture", (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                    1.3, (0, 255, 0), 3, cv2.LINE_AA)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(25) == ord('m'):
            break

    counter = 0
    while counter < class_images:
        ret, frame = cap.read()
        cv2.imshow("Frame", frame)
        cv2.waitKey(25)
        cv2.imwrite(os.path.join(class_dir, f'{counter}.jpg'), frame)
        counter += 1

cap.release()
cv2.destroyAllWindows()


In [22]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)

data_dir = './data'
data = []
labels = []

for dir in os.listdir(data_dir):
    for img_path in os.listdir(os.path.join(data_dir, dir)):
        auxdata = []

        img = cv2.imread(os.path.join(data_dir, dir, img_path))
        imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(imgrgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for lm in hand_landmarks.landmark:
                    auxdata.extend([lm.x, lm.y, lm.z])
            data.append(auxdata)
            labels.append(int(dir))

# Save processed data
with open('landmarks.pkl', 'wb') as f:
    pickle.dump({'data': data, 'labels': labels}, f)



In [35]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Load the data that was previously processed
with open('landmarks.pkl', 'rb') as f:
    data_dict = pickle.load(f)

data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

# Initialize the classifiers
RF_model = RandomForestClassifier()
KNN_model = KNeighborsClassifier()
LR_model = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence if needed

# Fit models on the training data and evaluate their accuracy
models = [RF_model, KNN_model, LR_model]
model_names = ['RandomForest', 'KNeighbors', 'LogisticRegression']
scores = []

for model in models:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

# Identify the best model based on accuracy
best_model_index = np.argmax(scores)
best_model_name = model_names[best_model_index]
best_model_score = scores[best_model_index]

# Train all models on the full dataset and save them
for model, name in zip(models, model_names):
    model.fit(data, labels)
    filename = f'{name}_model.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f'{name} model saved as {filename}')

# Print the best model and its accuracy
print(f'The best model is {best_model_name} with an accuracy score of {best_model_score}')


RandomForest model saved as RandomForest_model.pkl
KNeighbors model saved as KNeighbors_model.pkl
LogisticRegression model saved as LogisticRegression_model.pkl
The best model is RandomForest with an accuracy score of 1.0


In [40]:
import pickle
import cv2
import mediapipe as mp
import numpy as np

# Load the trained best model
model = pickle.load(open('LogisticRegression_model.pkl', 'rb'))

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.2)

# Label dictionary for A-Z
labels_dict = {i: chr(i + 65) for i in range(26)}

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret:
        break

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                x = lm.x
                y = lm.y
                z = lm.z  # Include the z-coordinate
                x_.append(x)
                y_.append(y)

            for lm in hand_landmarks.landmark:
                data_aux.append(lm.x - min(x_))  # Normalize x-coordinate
                data_aux.append(lm.y - min(y_))  # Normalize y-coordinate
                data_aux.append(lm.z)  # Include the z-coordinate without normalization

            # Draw the hand landmarks on the frame
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), circle_radius=2)
            )

        # Get the bounding box for the hand
        x1 = int(min(x_) * W) - 10
        x2 = int(max(x_) * W) + 10
        y1 = int(min(y_) * H) - 10
        y2 = int(max(y_) * H) + 10

        if len(data_aux) == 63:  # Ensure there are 63 features for prediction
            # Predict the class of the hand sign using the best model
            pred = model.predict([np.asarray(data_aux)])
            pred_char = labels_dict[int(pred[0])]  # Get the predicted character

            # Draw a rectangle and display the predicted character
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
            cv2.putText(frame, pred_char, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the frame with the predicted character
    cv2.imshow('Sign Detected', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [38]:
import pickle
import cv2
import mediapipe as mp
import numpy as np

# Load the trained best model
model = pickle.load(open('KNeighbors_model.pkl', 'rb'))

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.3)

# Label dictionary for A-Z
labels_dict = {i: chr(i + 65) for i in range(26)}

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret:
        break

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                x = lm.x
                y = lm.y
                z = lm.z  # Include the z-coordinate
                x_.append(x)
                y_.append(y)

            for lm in hand_landmarks.landmark:
                data_aux.append(lm.x - min(x_))  # Normalize x-coordinate
                data_aux.append(lm.y - min(y_))  # Normalize y-coordinate
                data_aux.append(lm.z)  # Include the z-coordinate without normalization

            # Draw the hand landmarks on the frame
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), circle_radius=2)
            )

        # Get the bounding box for the hand
        x1 = int(min(x_) * W) - 10
        x2 = int(max(x_) * W) + 10
        y1 = int(min(y_) * H) - 10
        y2 = int(max(y_) * H) + 10

        if len(data_aux) == 63:  # Ensure there are 63 features for prediction
            # Predict the class of the hand sign using the best model
            pred = model.predict([np.asarray(data_aux)])
            pred_char = labels_dict[int(pred[0])]  # Get the predicted character

            # Draw a rectangle and display the predicted character
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
            cv2.putText(frame, pred_char, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the frame with the predicted character
    cv2.imshow('Sign Detected', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [39]:
import pickle
import cv2
import mediapipe as mp
import numpy as np

# Load the trained best model
model = pickle.load(open('RandomForest_model.pkl', 'rb'))

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.3)

# Label dictionary for A-Z
labels_dict = {i: chr(i + 65) for i in range(26)}

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret:
        break

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                x = lm.x
                y = lm.y
                z = lm.z  # Include the z-coordinate
                x_.append(x)
                y_.append(y)

            for lm in hand_landmarks.landmark:
                data_aux.append(lm.x - min(x_))  # Normalize x-coordinate
                data_aux.append(lm.y - min(y_))  # Normalize y-coordinate
                data_aux.append(lm.z)  # Include the z-coordinate without normalization

            # Draw the hand landmarks on the frame
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), circle_radius=4),
                mp_drawing.DrawingSpec(color=(0, 0, 255), circle_radius=2)
            )

        # Get the bounding box for the hand
        x1 = int(min(x_) * W) - 10
        x2 = int(max(x_) * W) + 10
        y1 = int(min(y_) * H) - 10
        y2 = int(max(y_) * H) + 10

        if len(data_aux) == 63:  # Ensure there are 63 features for prediction
            # Predict the class of the hand sign using the best model
            pred = model.predict([np.asarray(data_aux)])
            pred_char = labels_dict[int(pred[0])]  # Get the predicted character

            # Draw a rectangle and display the predicted character
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
            cv2.putText(frame, pred_char, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the frame with the predicted character
    cv2.imshow('Sign Detected', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()